In [ ]:
, from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import re
import json

In [ ]:
def get_info():
  return {
    "description": "German Traffic Sign Detection Benchmark (GTSDB) dataset",
    "url": "https://benchmark.ini.rub.de/gtsdb_news.html",
    "version": "2.0",
    "year": 2013
  }
def get_licenses():
  return [
    {
        "url": "creativecommons.org/licenses/by-nc-sa/4.0/",
        "id": 1,
        "name": "CC BY-NC-SA 4.0"
    }
  ]

In [ ]:
def get_images(training=True):
    # Getting dataset root path
    if training:
        folder = '/content/drive/MyDrive/FullIJCNN2013/train'
    else:
        folder = '/content/drive/MyDrive/FullIJCNN2013/val'
    images = []
    id = 0
    for f in os.listdir(folder):
        if f.endswith('.jpg'):
          print(f)
          images.append(
          {
            "license": 1,
            "file_name": f,
            "height": 800,
            "width": 1360,
            "id": id
          })
          id += 1

    return images

In [ ]:
categories = []
categories.append({"supercategory": "prohibitory", "id": 0, "name": "speed limit 20"})
categories.append({"supercategory": "prohibitory", "id": 1, "name": "speed limit 30"})
categories.append({"supercategory": "prohibitory", "id": 2, "name": "speed limit 50"})
categories.append({"supercategory": "prohibitory", "id": 3, "name": "speed limit 60"})
categories.append({"supercategory": "prohibitory", "id": 4, "name": "speed limit 70"})
categories.append({"supercategory": "prohibitory", "id": 5, "name": "speed limit 80"})
categories.append({"supercategory": "other", "id": 6, "name": "restriction ends 80"})
categories.append({"supercategory": "prohibitory", "id": 7, "name": "speed limit 100"})
categories.append({"supercategory": "prohibitory", "id": 8, "name": "speed limit 120"})
categories.append({"supercategory": "prohibitory", "id": 9, "name": "no overtaking"})
categories.append({"supercategory": "prohibitory", "id": 10, "name": "no overtaking (trucks)"})
categories.append({"supercategory": "danger", "id": 11, "name": "priority at next intersection"})
categories.append({"supercategory": "other", "id": 12, "name": "priority road"})
categories.append({"supercategory": "other", "id": 13, "name": "give way"})
categories.append({"supercategory": "other", "id": 14, "name": "stop"})
categories.append({"supercategory": "prohibitory", "id": 15, "name": "no traffic both ways"})
categories.append({"supercategory": "prohibitory", "id": 16, "name": "no trucks"})
categories.append({"supercategory": "other", "id": 17, "name": "no entry"})
categories.append({"supercategory": "danger", "id": 18, "name": "danger"})
categories.append({"supercategory": "danger", "id": 19, "name": "bend left"})
categories.append({"supercategory": "danger", "id": 20, "name": "bend right"})
categories.append({"supercategory": "danger", "id": 21, "name": "bend"})
categories.append({"supercategory": "danger", "id": 22, "name": "uneven road"})
categories.append({"supercategory": "danger", "id": 23, "name": "slippery road"})
categories.append({"supercategory": "danger", "id": 24, "name": "road narrows"})
categories.append({"supercategory": "danger", "id": 25, "name": "construction"})
categories.append({"supercategory": "danger", "id": 26, "name": "traffic signal"})
categories.append({"supercategory": "danger", "id": 27, "name": "pedestrian crossing"})
categories.append({"supercategory": "danger", "id": 28, "name": "school crossing"})
categories.append({"supercategory": "danger", "id": 29, "name": "cycles crossing"})
categories.append({"supercategory": "danger", "id": 30, "name": "snow"})
categories.append({"supercategory": "danger", "id": 31, "name": "animals"})
categories.append({"supercategory": "other", "id": 32, "name": "restriction ends"})
categories.append({"supercategory": "mandatory", "id": 33, "name": "go right"})
categories.append({"supercategory": "mandatory", "id": 34, "name": "go left"})
categories.append({"supercategory": "mandatory", "id": 35, "name": "go straight"})
categories.append({"supercategory": "mandatory", "id": 36, "name": "go right or straight"})
categories.append({"supercategory": "mandatory", "id": 37, "name": "go left or straight"})
categories.append({"supercategory": "mandatory", "id": 38, "name": "keep right"})
categories.append({"supercategory": "mandatory", "id": 39, "name": "keep left"})
categories.append({"supercategory": "mandatory", "id": 40, "name": "roundabout"})
categories.append({"supercategory": "other", "id": 41, "name": "restriction ends (overtaking)"})
categories.append({"supercategory": "other", "id": 42, "name": "restriction ends (overtaking (trucks))"})

In [ ]:
def get_segmentation(upper_left_x, upper_left_y, lower_right_x, lower_right_y):
    # Vertices are: 
    # Upper left  -> both x y given
    # Upper right -> x is the same as lower right; y is the same as upper left
    # Lower left  -> x is the same as upper left;  y is the same as lower right
    # Lower right -> both x y given
    # Coordinates are counted from the upper left corner of the pic

    return [[
        upper_left_x, upper_left_y,
        lower_right_x, upper_left_y,
        upper_left_x, lower_right_y,
        lower_right_x, lower_right_y
    ]]


def get_area(upper_left_x, upper_left_y, lower_right_x, lower_right_y):
    # Area is width * height
    # Width is lower_right_x - upper_left_x (right X - left X)
    # Height is upper_left_y - lower_right_y (upper Y - lower Y)
    # Height is negative because of the coordinates!

    return (lower_right_x - upper_left_x)*(-(upper_left_y - lower_right_y))


def get_bbox(upper_left_x, upper_left_y, lower_right_x, lower_right_y):
    # The COCO bounding box format is [top left x position, top left y position, width, height]
    # Top left x position is upper_left_x
    # Top left y position is upper_left_y
    # Width is lower_right_x - upper_left_x (right X - left X)
    # Height is upper_left_y - lower_right_y (upper Y - lower Y)
    # Height is negative because of the coordinates!

    return [upper_left_x, upper_left_y, (lower_right_x - upper_left_x), (-(upper_left_y - lower_right_y))] 


In [ ]:
def get_annotations(images, categories, training=True):
    # Getting dataset root path
    if training:
        df = pd.read_csv('/content/drive/MyDrive/FullIJCNN2013/gt_train.txt', 
            index_col=False)
    else:
        df = pd.read_csv('/content/drive/MyDrive/FullIJCNN2013/gt_test.txt', 
            index_col=False)
        
    # Converting the dataframe into a list of lists
    data = df.values.tolist()

    # Each element in data is a 6-element list with the following fields:
    # 0. Filename
    # 1. leftCol
    # 2. topRow
    # 3. rightCol
    # 4. bottomRow
    # 5. ClassID

    annotations = []
    total = len(data)
    for i, d in enumerate(data, 1):

        # Getting image ID from images based on filename (d[0])
        # Since filename has the format dayTrain/dayClip1--00347.jpg we need to strip from the slash
        filename = d[0]
        image_id = [i["id"] for i in images if i["file_name"].endswith(filename)][0]

        # Getting category ID from categories based on name (d[1])
        category_id = d[5]

        segmentation = get_segmentation(d[1], d[2], d[3], d[4])
        area         = get_area(d[1], d[2], d[3], d[4])
        bbox         = get_bbox(d[1], d[2], d[3], d[4])

        annotations.append({
            "segmentation": segmentation, 
            "area": area,
            "iscrowd": 0,
            "image_id": image_id,
            "bbox": bbox,
            "category_id": category_id,
            "id": i 
        })
        print(annotations[-1])
        print(f'Processed {i} of {total}')
    return annotations

In [ ]:
info = get_info()
licenses = get_licenses()
training_images = get_images(training=True)
test_images     = get_images(training=False)
training_annotations = get_annotations(training_images, 
        categories, training=True)
test_annotations     = get_annotations(test_images, 
        categories, training=False)
training_data = {
        "info": info,
        "licenses": licenses,
        "images": training_images,
        "annotations": training_annotations,
        "categories": categories
    }

test_data = {
        "info": info,
        "licenses": licenses,
        "images": test_images,
        "annotations": test_annotations,
        "categories": categories
    }

with open('/content/instances_train2017.json', 'w', encoding='utf-8') as f:
    json.dump(training_data, f, ensure_ascii=False, indent=4)
with open('/content/instances_val2017.json', 'w', encoding='utf-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=4)


00790.jpg
00793.jpg
00797.jpg
00795.jpg
00799.jpg
00798.jpg
00792.jpg
00787.jpg
00789.jpg
00796.jpg
00794.jpg
00791.jpg
00788.jpg
00785.jpg
00782.jpg
00777.jpg
00780.jpg
00773.jpg
00784.jpg
00775.jpg
00776.jpg
00783.jpg
00778.jpg
00786.jpg
00779.jpg
00774.jpg
00781.jpg
00760.jpg
00772.jpg
00762.jpg
00769.jpg
00759.jpg
00765.jpg
00761.jpg
00768.jpg
00771.jpg
00770.jpg
00763.jpg
00764.jpg
00766.jpg
00767.jpg
00752.jpg
00757.jpg
00746.jpg
00751.jpg
00758.jpg
00747.jpg
00749.jpg
00750.jpg
00753.jpg
00748.jpg
00756.jpg
00755.jpg
00754.jpg
00737.jpg
00736.jpg
00740.jpg
00743.jpg
00731.jpg
00739.jpg
00735.jpg
00744.jpg
00745.jpg
00734.jpg
00733.jpg
00732.jpg
00742.jpg
00738.jpg
00741.jpg
00717.jpg
00725.jpg
00726.jpg
00728.jpg
00730.jpg
00722.jpg
00724.jpg
00727.jpg
00729.jpg
00720.jpg
00721.jpg
00718.jpg
00719.jpg
00723.jpg
00713.jpg
00703.jpg
00715.jpg
00710.jpg
00711.jpg
00709.jpg
00708.jpg
00705.jpg
00704.jpg
00706.jpg
00714.jpg
00716.jpg
00712.jpg
00707.jpg
00701.jpg
00695.jpg
00694.jpg
